In [1]:
# Allows for further data manipulation and analysis
import pandas as pd
from pandas_datareader import data as web # Reads stock data
import yfinance as yf
%matplotlib inline

# For defining dates
import datetime as dt

# Read from sql
import sqlite3

# Default Values

In [2]:
# Define path to files
PATH = "/Users/jwiegand/Dev/jrwiegand/data/files"

# Start date defaults
S_YEAR = 2021
S_MONTH = 6
S_DAY = 1
S_DATE_STR = f"{S_YEAR}-{S_MONTH}-{S_DAY}"
S_DATE_DATETIME = dt.datetime(S_YEAR, S_MONTH, S_DAY)

# Start date defaults
E_YEAR = 2024
E_MONTH = 6
E_DAY = 1
E_DATE_STR = f"{E_YEAR}-{E_MONTH}-{E_DAY}"
E_DATE_DATETIME = dt.datetime(E_YEAR, E_MONTH, E_DAY)


# Manually Download the Screener CSV from Nasdaq
# https://www.nasdaq.com/market-activity/stocks/screener
CSV_DATA_FILE_NAME = "/nasdaq_screener_1717356628553.csv"

# Stocks that were not downloaded
stocks_not_downloaded = []

# Save Nasdaq Symbols to SQL

In [3]:
def save_symbols_to_sql(csv_file, db_file, table_name):
    # Read the CSV file into a pandas DataFrame
    df = pd.read_csv(csv_file)

    # Connect to the SQLite database (creates it if it doesn't exist)
    connection = sqlite3.connect(db_file)

    df.to_sql("stocks", connection, if_exists="replace", index=True)

    connection.close()
    print(f"Data from {csv_file} successfully imported into {db_file} in table {table_name}")

# Returns a Named Column Data from a CSV

In [4]:
def get_column_from_csv(file, col_name):
    # Try to get the file and if it doesn't exist issue a warning
    try:
        df = pd.read_csv(file)
        df[col_name] = df[col_name].astype(str)
        df[col_name] = df[col_name].apply(str)
    except FileNotFoundError:
        print("File Doesn't Exist")
    else:
        return df[col_name].astype("string")

# Function that Saves Stock Data to CSV

In [5]:
# Function that gets a dataframe by providing a ticker and starting date
def save_to_csv_from_yahoo(folder, stock_symbol, start, end):
    try:
        print("Get Data for: ", stock_symbol)

        yf.pdr_override()

        # Reads data into a dataframe
        df = web.DataReader(stock_symbol, start, end)["Adj Close"]

        # Save data to a CSV file
        df.to_csv(folder + stock_symbol + ".csv")
    except Exception as ex:
        print(ex)
        stocks_not_downloaded.append(stock_symbol)
        print("Could Not Get Data for :", stock_symbol)

# Test Receiving Stock Tickers

In [6]:
save_symbols_to_sql(PATH + CSV_DATA_FILE_NAME, PATH + "/stocks.db", "stocks")

Data from /Users/jwiegand/Dev/jrwiegand/data/files/nasdaq_screener_1717356628553.csv successfully imported into /Users/jwiegand/Dev/jrwiegand/data/files/stocks.db in table stocks


# Get 5 Years of Data for all the Stocks

In [8]:
connection = sqlite3.connect(PATH + "/stocks.db")
cursor = connection.cursor()
cursor.execute("SELECT Symbol FROM stocks")
rows = cursor.fetchall()

for row in rows:
  symbol = row[0]
  save_to_csv_from_yahoo(PATH + "/stocks/", symbol, S_DATE_DATETIME, E_DATE_DATETIME)

print("Finished")
print("Stocks Not Downloaded: ", stocks_not_downloaded)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

Get Data for:  A
Get Data for:  AA
Get Data for:  AACG
Get Data for:  AACI
Get Data for:  AACIU
Get Data for:  AACIW



[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['AACIW']: YFInvalidPeriodError("%ticker%: Period 'max' is invalid, must be one of ['1d', '5d']")
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


Get Data for:  AACT
Get Data for:  AADI
Get Data for:  AAGR
Get Data for:  AAL
Get Data for:  AAMC
Get Data for:  AAME


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


...

Get Data for:  ZVRA
Get Data for:  ZVSA
Get Data for:  ZWS


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


Get Data for:  ZYME
Get Data for:  ZYXI


[*********************100%%**********************]  1 of 1 completed

Finished
Stocks Not Downloaded:  ['AKO/A', 'AKO/B', 'BF/A', 'BF/B', 'BIO/B', 'BRK/A', 'BRK/B', 'CRD/A', 'CRD/B', 'HEI/A', 'HVT/A', None, 'WSO/B']
